In [5]:
# === Cell 1: CONFIG ===
# Edit ONLY the values in the "USER SETTINGS" block.
# This cell intentionally contains placeholders so you don't leak any secrets or local paths in version control.

from pathlib import Path
import os

# ---------- USER SETTINGS (EDIT THESE) ----------
# Project root folder. If you're running the notebook from the repo root, you can leave as "."
PROJECT_ROOT = Path(".").resolve()

# Paths to your district boundaries
# 👉 Update these to point to your own files
PATH_DISTRICTS_GPKG    = PROJECT_ROOT / "PATH" / "TO" / "districts.gpkg"           # <--- CHANGE ME
PATH_DISTRICTS_GEOJSON = PROJECT_ROOT / "PATH" / "TO" / "bezirksgrenzen.geojson"   # <--- CHANGE ME

# Output folder + file for the results
OUT_DIR  = PROJECT_ROOT / "OUTPUT" / "FOLDER"   # <--- CHANGE ME
OUT_CSV  = OUT_DIR / "district_travel_times.csv"

# Landmarks (default list — edit as needed)
LANDMARKS = [
    {"name": "Alexanderplatz",                   "lat": 52.521918, "lon": 13.413215},
    {"name": "Berlin Brandenburg Airport (BER)", "lat": 52.366667, "lon": 13.503333},
    {"name": "Berlin Hauptbahnhof",              "lat": 52.525083, "lon": 13.369402},
    {"name": "Charité Hospital",                 "lat": 52.530407, "lon": 13.379809},
]

# Modes to compute
MODES = ["driving", "walking", "transit"]

# Batch size for Distance Matrix (safe default: 10x10 => 100 elements per call)
BATCH_ORIGINS = 10
BATCH_DESTS   = 10

# 🔑 API key (set your Google Maps API key here!)
API_KEY = "YOUR_API_KEY_HERE"   # <-- replace with your key
# ------------------------------------------------

# ---------- DO NOT EDIT BELOW ----------
# Resolve which districts file to use
if PATH_DISTRICTS_GPKG.exists():
    DISTRICTS_PATH = PATH_DISTRICTS_GPKG
elif PATH_DISTRICTS_GEOJSON.exists():
    DISTRICTS_PATH = PATH_DISTRICTS_GEOJSON
else:
    raise FileNotFoundError(
        f"Could not find districts file.\nChecked:\n - {PATH_DISTRICTS_GPKG}\n - {PATH_DISTRICTS_GEOJSON}"
    )

# Create output dir if missing
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Summary printout
print("✓ Config loaded")
print(f"Project root:           {PROJECT_ROOT}")
print(f"Districts file:         {DISTRICTS_PATH}")
print(f"Output CSV:             {OUT_CSV}")
print(f"Landmarks:              {[lm['name'] for lm in LANDMARKS]}")
print(f"Modes:                  {MODES}")
print(f"Batching (O x D):       {BATCH_ORIGINS} x {BATCH_DESTS}")
print(f"API key set?            {API_KEY.startswith('AIza') if API_KEY else False}")

✓ Config loaded
Project root:           /Users/giovanigoltara/Documents/webeet/layered-district-deep-dive-analysis/Giovani
Districts file:         /Users/giovanigoltara/Documents/webeet/layered-district-deep-dive-analysis/Giovani/scripts/districts.gpkg
Output CSV:             /Users/giovanigoltara/Documents/webeet/layered-district-deep-dive-analysis/Giovani/scripts/out/district_travel_times.csv
Landmarks:              ['Alexanderplatz', 'Berlin Brandenburg Airport (BER)', 'Berlin Hauptbahnhof', 'Charité Hospital']
Modes:                  ['driving', 'walking', 'transit']
Batching (O x D):       10 x 10
API key set?            True


In [6]:
# === Cell 2: LOAD DISTRICTS & BUILD ORIGINS (CENTROIDS) ===
import geopandas as gpd
import pandas as pd

# 1) Read the districts layer (gpkg or geojson chosen in Cell 1)
gdf = gpd.read_file(DISTRICTS_PATH)

# 2) Try to detect a usable district name column
#    Adjust this list if your file uses a different field for the district name.
CAND_NAME_COLS = ["district", "bezirk", "name", "GEN", "bezirk_name", "NAME", "NAME_1"]
name_col = next((c for c in CAND_NAME_COLS if c in gdf.columns), None)

if name_col is None:
    # Fallback: create a synthetic name if none found
    gdf["district"] = [f"district_{i}" for i in range(len(gdf))]
    name_col = "district"

# 3) Compute centroids safely: project → centroid → back to WGS84
#    3857 works fine here; if you prefer, you can swap for Berlin’s EPSG:25833.
gdf_proj = gdf.to_crs(3857)
gdf_proj["centroid"] = gdf_proj.geometry.centroid
gdf_cent = gdf_proj.set_geometry("centroid").to_crs(4326)

# 4) Extract origin points (lat/lon) per district
gdf_cent["lat"] = gdf_cent.geometry.y
gdf_cent["lon"] = gdf_cent.geometry.x

origins_df = gdf_cent[[name_col, "lat", "lon"]].rename(columns={name_col: "district"}).copy()

# Optional: sort by name for nicer outputs
origins_df = origins_df.sort_values("district").reset_index(drop=True)

# 5) Preview
display_cols = ["district", "lat", "lon"]
print(f"✓ Loaded {len(origins_df)} districts from {DISTRICTS_PATH.name}")
origins_df.head(len(origins_df) if len(origins_df) < 20 else 10)[display_cols]

✓ Loaded 12 districts from districts.gpkg


,district,lat,lon
0,Charlottenburg-Wilmersdorf,52.501053,13.270254
1,Friedrichshain-Kreuzberg,52.503816,13.428237
2,Lichtenberg,52.534952,13.509769
3,Marzahn-Hellersdorf,52.521140,13.578828
4,Mitte,52.532905,13.365967
5,Neukölln,52.441102,13.454957
6,Pankow,52.597269,13.440285
7,Reinickendorf,52.595694,13.291343
8,Spandau,52.525845,13.178832
9,Steglitz-Zehlendorf,52.434854,13.239083


In [8]:
# === Cell 3: CALL GOOGLE DISTANCE MATRIX ===
# One-shot run: origins = district centroids, destinations = LANDMARKS, modes = MODES

import time, datetime as dt
import pandas as pd
import googlemaps
from pytz import timezone

# ---- 0) Sanity checks ----
if not API_KEY or not API_KEY.startswith("AIza"):
    raise ValueError("API_KEY not set or malformed. Edit Cell 1 and paste your real key.")

if origins_df is None or origins_df.empty:
    raise ValueError("origins_df is empty. Run Cell 2 successfully before this cell.")

# ---- 1) Build client ----
gmaps = googlemaps.Client(key=API_KEY)

# ---- 2) Prepare inputs ----
destinations = [{"name": lm["name"], "lat": lm["lat"], "lon": lm["lon"]} for lm in LANDMARKS]

# Departure time = now (must be current or future)
CITY_TZ = timezone("Europe/Berlin")
DEPARTURE_TIME = int(dt.datetime.now(CITY_TZ).timestamp())

# ---- 3) Small helpers ----
def batched(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

def dm_request(origins_batch, dests_batch, mode, departure_time=None, max_retries=4):
    """Call Distance Matrix with gentle retries."""
    req = {
        "origins": [(o["lat"], o["lon"]) for o in origins_batch],
        "destinations": [(d["lat"], d["lon"]) for d in dests_batch],
        "mode": mode,
    }
    if departure_time and mode in ("driving", "transit"):
        req["departure_time"] = departure_time

    for i in range(max_retries + 1):
        try:
            return gmaps.distance_matrix(**req)
        except Exception as e:
            if i == max_retries:
                raise
            sleep = 2 ** i
            print(f"[{mode}] Error '{e}'. Retry in {sleep}s...")
            time.sleep(sleep)

# ---- 4) Build origin/destination lists for batching ----
origins_list = origins_df.to_dict(orient="records")
# Rename keys to match helper
origins_list = [{"district": o["district"], "lat": o["lat"], "lon": o["lon"]} for o in origins_list]

# ---- 5) Fire requests (batched) ----
records = []
total_elements = len(origins_list) * len(destinations) * len(MODES)
print(f"Starting Distance Matrix run for {len(origins_list)} origins × {len(destinations)} destinations × {len(MODES)} modes = {total_elements} elements.")

element_counter = 0
for mode in MODES:
    for o_batch in batched(origins_list, BATCH_ORIGINS):
        for d_batch in batched(destinations, BATCH_DESTS):
            res = dm_request(o_batch, d_batch, mode, departure_time=DEPARTURE_TIME)
            rows = res.get("rows", [])
            for oi, row in enumerate(rows):
                origin_meta = o_batch[oi]
                for di, elem in enumerate(row.get("elements", [])):
                    dest_meta = d_batch[di]
                    element_counter += 1
                    rec = {
                        "district": origin_meta["district"],
                        "origin_lat": origin_meta["lat"],
                        "origin_lon": origin_meta["lon"],
                        "landmark": dest_meta["name"],
                        "dest_lat": dest_meta["lat"],
                        "dest_lon": dest_meta["lon"],
                        "mode": mode,
                        "api_status": elem.get("status"),
                        "distance_m": None,
                        "duration_sec": None,
                        "duration_in_traffic_sec": None,
                        "departure_time_iso": dt.datetime.fromtimestamp(DEPARTURE_TIME, tz=CITY_TZ).isoformat()
                    }
                    if elem.get("status") == "OK":
                        if "distance" in elem: rec["distance_m"] = elem["distance"]["value"]
                        if "duration" in elem: rec["duration_sec"] = elem["duration"]["value"]
                        if "duration_in_traffic" in elem: rec["duration_in_traffic_sec"] = elem.get("duration_in_traffic", {}).get("value")
                    records.append(rec)
            print(f"  - {mode}: {element_counter}/{total_elements} elements processed...")

results_df = pd.DataFrame(records)
print("✓ Distance Matrix calls complete.")
print(results_df.head(8))
print(f"Rows: {len(results_df)}")

Starting Distance Matrix run for 12 origins × 4 destinations × 3 modes = 144 elements.
  - driving: 40/144 elements processed...
  - driving: 48/144 elements processed...
  - walking: 88/144 elements processed...
  - walking: 96/144 elements processed...
  - transit: 136/144 elements processed...
  - transit: 144/144 elements processed...
✓ Distance Matrix calls complete.
                     district  origin_lat  origin_lon  \
0  Charlottenburg-Wilmersdorf   52.501053   13.270254   
1  Charlottenburg-Wilmersdorf   52.501053   13.270254   
2  Charlottenburg-Wilmersdorf   52.501053   13.270254   
3  Charlottenburg-Wilmersdorf   52.501053   13.270254   
4    Friedrichshain-Kreuzberg   52.503816   13.428237   
5    Friedrichshain-Kreuzberg   52.503816   13.428237   
6    Friedrichshain-Kreuzberg   52.503816   13.428237   
7    Friedrichshain-Kreuzberg   52.503816   13.428237   

                           landmark   dest_lat   dest_lon     mode api_status  \
0                    Alexander

In [10]:
# === Cell 4: SAVE RESULTS & CREATE PIVOTS ===
import pandas as pd
import numpy as np
from pathlib import Path

# 0) Checks
if 'results_df' not in globals() or results_df.empty:
    raise ValueError("results_df is empty. Run Cell 3 first.")

# 1) Clean & enrich for readability
df = results_df.copy()

# Convert to minutes / kilometers
df["duration_min"] = (df["duration_sec"] / 60).round(1)
df["distance_km"] = (df["distance_m"] / 1000).round(2)

# Keep essential cols in a nice order
cols = [
    "district", "landmark", "mode",
    "duration_sec", "duration_min",
    "distance_m", "distance_km",
    "api_status", "departure_time_iso",
    "origin_lat", "origin_lon", "dest_lat", "dest_lon"
]
df = df[cols]

# 2) Save full long table
OUT_CSV.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(OUT_CSV, index=False)
print(f"✓ Wrote long table: {OUT_CSV}  (rows={len(df)})")

# 3) Pivot for dashboard: median minutes by district × landmark × mode
pivot = (
    df[df["api_status"] == "OK"]
    .pivot_table(
        index=["district", "landmark"],
        columns="mode",
        values="duration_min",
        aggfunc="median"
    )
    .reset_index()
)

pivot_path = OUT_DIR / "district_landmark_travel_minutes_pivot.csv"
pivot.to_csv(pivot_path, index=False)
print(f"✓ Wrote pivot (median minutes): {pivot_path}  (rows={len(pivot)})")

# 4) Compact KPIs per district: median across all landmarks (one row per district)
kpis = (
    df[df["api_status"] == "OK"]
    .groupby(["district", "mode"], as_index=False)["duration_min"].median()
    .pivot(index="district", columns="mode", values="duration_min")
    .reset_index()
    .rename_axis(None, axis=1)
)

kpis_path = OUT_DIR / "district_kpis_by_mode.csv"
kpis.to_csv(kpis_path, index=False)
print(f"✓ Wrote district KPIs: {kpis_path}  (rows={len(kpis)})")

# 5) Quick preview
print("\nPreview — KPIs (median minutes by mode per district):")
display_cols = ["district"] + [c for c in ["driving","walking","transit"] if c in kpis.columns]
kpis.sort_values("district").head(20)[display_cols]

✓ Wrote long table: /Users/giovanigoltara/Documents/webeet/layered-district-deep-dive-analysis/Giovani/scripts/out/district_travel_times.csv  (rows=144)
✓ Wrote pivot (median minutes): /Users/giovanigoltara/Documents/webeet/layered-district-deep-dive-analysis/Giovani/scripts/out/district_landmark_travel_minutes_pivot.csv  (rows=48)
✓ Wrote district KPIs: /Users/giovanigoltara/Documents/webeet/layered-district-deep-dive-analysis/Giovani/scripts/out/district_kpis_by_mode.csv  (rows=12)

Preview — KPIs (median minutes by mode per district):


,district,driving,walking,transit
0,Charlottenburg-Wilmersdorf,29.25,142.95,34.80
1,Friedrichshain-Kreuzberg,20.80,76.15,28.85
2,Lichtenberg,28.40,136.90,50.30
3,Marzahn-Hellersdorf,40.70,220.80,50.40
4,Mitte,12.00,37.95,20.10
5,Neukölln,30.70,171.40,46.80
6,Pankow,29.55,144.50,46.15
7,Reinickendorf,30.45,165.80,50.65
8,Spandau,37.90,224.10,45.05
9,Steglitz-Zehlendorf,36.70,230.00,53.80


In [13]:
# === Cell 4C: District × Landmark table (median minutes, relabeled modes) ===
import pandas as pd

if 'results_df' not in globals() or results_df.empty:
    raise ValueError("results_df is empty. Run Cell 3 first.")

# 1) Keep only successful results and compute minutes
df_ok = results_df[results_df["api_status"] == "OK"].copy()
df_ok["duration_min"] = (df_ok["duration_sec"] / 60).round(1)

# 2) Pivot
table = (
    df_ok.pivot_table(
        index=["district", "landmark"],
        columns="mode",
        values="duration_min",
        aggfunc="median"
    )
    .reset_index()
)

# 3) Relabel columns
rename_map = {
    "driving": "car_min",
    "walking": "walk_min",
    "transit": "pt_min"
}
table = table.rename(columns=rename_map)

# 4) Ensure column order
cols = ["district", "landmark"] + [c for c in ["car_min", "walk_min", "pt_min"] if c in table.columns]
table = table[cols]

# 5) Order landmarks like in LANDMARKS
landmark_order = [lm["name"] for lm in LANDMARKS]
table["landmark"] = pd.Categorical(table["landmark"], categories=landmark_order, ordered=True)
table = table.sort_values(["district", "landmark"]).reset_index(drop=True)

# 6) Save
out_path = OUT_DIR / "district_landmark_mode_minutes.csv"
table.to_csv(out_path, index=False)
print(f"✓ Wrote: {out_path}  (rows={len(table)})")

# 7) Preview
table.head(20)

✓ Wrote: /Users/giovanigoltara/Documents/webeet/layered-district-deep-dive-analysis/Giovani/scripts/out/district_landmark_mode_minutes.csv  (rows=48)


mode,district,landmark,car_min,walk_min,pt_min
0,Charlottenburg-Wilmersdorf,Alexanderplatz,30.7,153.6,31.5
1,Charlottenburg-Wilmersdorf,Berlin Brandenburg Airport (BER),27.9,420.6,80.3
2,Charlottenburg-Wilmersdorf,Berlin Hauptbahnhof,22.7,117.6,21.6
3,Charlottenburg-Wilmersdorf,Charité Hospital,30.6,132.3,38.1
4,Friedrichshain-Kreuzberg,Alexanderplatz,12.2,37.6,21.6
5,Friedrichshain-Kreuzberg,Berlin Brandenburg Airport (BER),26.3,286.2,43.2
6,Friedrichshain-Kreuzberg,Berlin Hauptbahnhof,22.1,79.0,22.2
7,Friedrichshain-Kreuzberg,Charité Hospital,19.5,73.3,35.5
8,Lichtenberg,Alexanderplatz,19.9,97.9,41.0
9,Lichtenberg,Berlin Brandenburg Airport (BER),39.8,329.3,82.4
